In [ ]:
%matplotlib nbagg
# Libraries
import scipy as sc
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import matplotlib
from   IPython.display import Markdown, display
#
from   repropy import Context, Meta, Tagged
import analysis
import analysis.plots as plot
from analysis.numass.meta import *
#
import unfolding as lib

In [ ]:
meta = Meta(
    dataset = dataset_2011 + Meta(
        drop_last=8,
    ),
    basis = Meta(
        oversample = 2,
    ),
    transmission = 'linear',
    omega        = [Tagged('OmegaIntSpec', deg=2, equalize=True)],
    # --
    mcmc = Meta(
        phiPrior   = "positive",
#        phiPrior   = "any",
#        alphaPrior = None,
#        alphaPrior = 0.00625226,
        alphaPrior = 0.00625226**2,
        SEED   = 34552,
        nCores = 8,
        draws  = 4000,      
    )
)
ctx = Context(meta, store_dir='../store')
am  = analysis.mcmc.MCMC(ctx)
aa  = am.aa
aPosterior = aa.aPost.value()

# Model with positivity

In [ ]:
MAP   = am.MAP.value()
trace = am.trace.value()

In [ ]:
pm.summary(trace).round(3)

In [ ]:
pm.traceplot(trace)
None

In [ ]:
fig = plot.plot_alpha(aa.unfold.value(), aPosterior)
dens,bins = np.histogram(trace['alpha'], bins=200, density=True)
dens = np.log(dens)
dens = dens - np.max(dens)
fig.axes[0].set_ylim((-2, 0.1))
plt.bar(x=bins[:-1], height=dens+2, width=np.diff(bins), bottom=-2, align="edge", fc=(0.1,0.1,0.8,0.3))
plt.semilogx()
None

In [ ]:
plot.plot_measurements_mcmc(aa, trace)

In [ ]:
fig = plot.plot_deconvolved_mcmc(aa, trace, xlim=(0,60))
None

In [ ]:
plot.plot_correlations_matrix( np.cov(trace['phi'], rowvar=False))

In [ ]:
fig.savefig('MCMC-deconvolved.svg')

In [ ]:
v = np.average(trace['phi'], axis=0)
#v = MAP['phi']
np.dot(np.dot(v, aa.unfold.value().omegas[0].mat),v)

In [ ]:
plot.plot_deconvolved_mcmc(aa, trace, xlim=(0,60))
plot.plot_deconvolved_mcmc(aa, {'phi':[MAP['phi']]}, only_mean=True, add=False, colstr='k')
plt.grid()
None

In [ ]:
for i in trace['phi']:
    print(len(i))
    break

In [ ]:
def plot_deconvolved_mcmc2(aa, trace,  xlim=None, ylim=None):
    basis = aa.basis.value()
    vs   = np.linspace(aa.dat.value()['vs'].values[0], aa.dat.value()['vs'].values[-1], 500)
    # mean value
    coef = np.average(trace['phi'], axis=0)
    phi  = lib.PhiVec(coef, basis)
    ys   = phi(vs)
    # calculate quantiles    
    bVals = [aa.basis.value()[i](vs) for i in range(len(aa.basis.value()))]
    bVals = np.asarray(bVals)
    quant = np.quantile(np.dot(trace['phi'], bVals), [0.16, 0.84], axis=0)
    yLow  = quant[0]
    yHi   = quant[1]
    #
    fig = plt.figure()
    plt.grid()
    gunE = aa.dat.meta().el_gun_E
    plt.plot(gunE - vs, ys, 'r')
    plt.plot(gunE - aa.dat.value()['vs'].values, phi(aa.dat.value()['vs'].values), 'g.')
    plt.fill_between(gunE - vs, yLow, yHi, color='b', alpha=0.5)  
    if xlim is not None:
        fig.axes[0].set_xlim(xlim)
        if ylim is None:
            x1,x2 = xlim
            idx   = ((gunE - vs) >= x1) & ((gunE - vs) <= x2)
            y1    = np.min(np.asarray(yLow)[idx])
            y2    = np.max(np.asarray(yHi)[idx])
            delta = (y2 - y1) * 0.05
            ylim  = (y1 - delta, y2 + delta)
    if ylim is not None:
        fig.axes[0].set_ylim(ylim)
    return fig

#
fig = plot_deconvolved_mcmc2(aa, trace, xlim=(0,12))
None

In [ ]:
plt.title('Deconvosution without positivity')

In [ ]:
fig.savefig("MCMC-any-zoom.svg")

Оценки негладкости функции

In [ ]:
om  = aa.unfold.value().omegas[0].mat
np.dot(np.dot( trace['phi'], om), trace['phi'].T).shape
oms = np.sum(np.dot(trace['phi'], om) * trace['phi'], axis=1)

plt.figure()
plt.hist(oms, bins=200)
None

In [ ]:
fig = plt.figure()
plt.hist(trace['phi'][:,90], bins=100 )
plt.grid()
plt.title("Distribution of φ[i]")
#fig = plt.figure()
#plt.hist( )
fig.savefig("MCMC-phi-apost.svg")
None